In [3]:
import numpy as np
import qutip as qt
import itertools as it
import multiprocessing as m
import os
import time
import matplotlib.pyplot as plt
from scipy.special import factorial
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
from operator import xor
import cvxpy     as cp
import scipy     as sci
import numpy.matlib
import scipy.io
import time


In [4]:
ide    = qt.identity(2)
sx     = qt.sigmax()
sy     = qt.sigmay()
sz     = qt.sigmaz()
si     = qt.identity(2)
v1, d1 = qt.Qobj.eigenstates(sx)
v2, d2 = qt.Qobj.eigenstates(sy)
v3, d3 = qt.Qobj.eigenstates(sz)

np.set_printoptions(precision=3,linewidth = np.inf)




def hamiltonian(pauli, n):
        """TODO: Docstring for hamiltonian.

        :pauli: The to local pauli matrix of choice
        :returns: the local hamiltonian for a given pauli matrix
        
        """
        return sum( [qt.tensor([si if i !=j else pauli for i in range(n) ]) for j in range(n)])


def unitary(n, alpha):
        """TODO: Docstring for unitary.

        :n: numer of qubits
        :alpha: the parameters we wish to estimate for pauli x,y,z collective operators 
        :returns: The unitary for a local 3D collective spin hamiltonian

        """
        Sx, Sy, Sz = [hamiltonian(sx,n),hamiltonian(sy,n),hamiltonian(sz,n)]
        H          = alpha[0]*Sx + alpha[1]*Sy + alpha[2]*Sz
        U          = qt.Qobj.expm(-1j*H)
        return U
    



In [5]:

#makes life easier to define the paulis and identity here
ide = qt.identity(2)
sx = qt.sigmax()
sy = qt.sigmay()
sz = qt.sigmaz()

#this is just a set of useful functions that are used through the calculation

def kron(lis,a): # returns the kronencher product of a list of matricies. a = len(list)
    if a == 1:
        out = np.kron(lis[0],lis[1])
    else:
        out = np.kron(kron(lis, a-1),lis[a])
    return out

def pauli_i(phase): # returns a single qubit rotation from a list of three phases
	lis = qt.Qobj.expm(-1j*(phase[0]*sx + phase[1]*sz + phase[2]*sy))
	return lis

def cnot_big(n, indexes): #takes the generated indexes[a, b,c] where c referes to cnot being on or off, a is the control and b is the target. n is the numer of qubits of the system
	if indexes[2] == 0:
		return qt.tensor([ide for i in range(n)])
	else:
		return qt.cnot(n, indexes[0]-1, indexes[1]-1)

def matrix_mult(lis): #takes a list a matricies and multiplies them all together
	a = lis[0]
	if len(lis) > 1:
		for i in range(len(lis)-1):
			a = lis[i+1]*a
	return a



def kraus_set(n,gamma):
        """define a set of kraus operators for dephasing

        :n: number of qubits
        :gamma: strength of dephasing gamma belongs to interval [0,1]
        :returns: a set of kraus operators for a given dephasing strength

        """
        e0    = qt.Qobj([[1, 0], [0, np.sqrt(1-gamma)]])
        e1    = qt.Qobj([[0,0],[0, np.sqrt(gamma)]])
        kraus =[x for x in it.product([e0,e1], repeat = n)]
        out   =[]
        for i in range(len(kraus)):
                out.append(qt.tensor([kraus[i][j] for j in range(n) ]))
        return out



#here we start to deal with the circuit itself

class circuit: #generates circuit cells from http://cds.cern.ch/record/931003/files/0602174.pdf . n is the number of qubits
    def __init__(self, n, g, du, gamma, kraus, init = False):
        self.g = g
        self.n = n
        self.gamma  = gamma
        self.kraus = kraus
        if init == False:
            self.sq  =  [np.random.choice(2) for i in range(g*n*(n+1))]
            self.tq  =  [np.random.choice(2) for i in range(g*n*(n-1))]
            self.sqp =  [2*np.pi*np.random.random() for i in range(3*sum(self.sq))]
        else:
            self.sq, self.tq = init[0], init[1], 
            self.sqp =  [2*np.pi*np.random.random() for i in range(3*sum(self.sq))]

        #size of sq and sqp is 2*g*n*(n+1)
        #size of tq is g*n*(n-1)
        
        self.cnots = []
        
        count = 0 
        for i in range(g):
            for j in range(n):
                temp_c = qt.tensor([si]*n)
                for k in range(n):
                    if k != j:
                        #print(self.tq[count])
                        if self.tq[count] == 1:
                            #print('cnot',qt.cnot(self.n, j, k))
                            temp_c = qt.cnot(self.n, j, k)*temp_c
                            count += 1
                            self.cnots.append(temp_c)
                        else:
                            self.cnots.append(temp_c)
                            count += 1
        x0 = [np.random.random() for i in range(len(self.sqp))]
        if self.gamma == 0:
            res = minimize(circuit.func, x0, args=(self,du), method = "Nelder-Mead")
        else:
            res = minimize(circuit.func_noise, x0, args=(self,du))#, method = "Nelder-Mead")
        self.qfi = res.fun
        self.sqp = res.x
        print(res.message, res.status)
        #print(self.cnots)
    
    def unitary(self):
        count = 0
        c_count = 0
        p_c = 0
        phases = np.array_split(self.sqp, sum(self.sq)) #fix this if sum(self.sq) = 0
        #print(len(self.sq),sum(self.sq), self.sq)
        #print(phases)
        u = qt.tensor([si]*self.n)
        for i in range(self.g):
            for j in range(self.n+1):
                squt = []
                for k in range(self.n):
                    if self.sq[count] == 1:
                        #print(count)
                        h = phases[p_c][0]*sx + phases[p_c][1]*sy +phases[p_c][2]*sz
                        squt.append(qt.Qobj.expm(-1j*h))
                        p_c += 1
                    else:
                        squt.append(si)
                        #count += 1
                    count += 1
                    
                if j == self.n:
                    #print('final')
                    u = qt.tensor(squt)*u
                else:
                    #print(c_count)
                    #print(self.cnots[c_count],qt.tensor(squt))
                    u = self.cnots[c_count]*qt.tensor(squt)*u
                    c_count += 1
        return u

    
    def eig(rho, phi, n):
        """eigen decompostion, assuming parameter is ~0

        :rho: evolved state
        :returns: Eigvenvectors and tidied Eigenvalues

        """
             
        D, Vi = np.linalg.eigh(rho.full())
        D     = np.real(D)
        Vi    = Vi[:,::-1]
        D     = D[::-1]
        Vi    = qt.Qobj(Vi,dims= [[2]*n,[2]*n])


        rank = sum([D[i] > 1e-7 for i in range(2**n) ]) #sum([phi[i][0][0] != 0 for i in range(2**n)  ])
        D[rank:] = 0
        Dclean = D[:rank]
        #print(rank, D, Dclean)
        return D, Vi, Dclean, rank

    
    
    def func_noise(phases, *args):
        self, du = args[0], args[1]
        self.sqp = phases
        zo       = qt.basis(2)
        u        = circuit.unitary(self)
        #qf_mat   = np.zeros((3,3))
        phi      = qt.tensor([zo]*self.n)
        phi_0    = u*phi
        rho_0    = phi_0*phi_0.dag()
        rho      = sum([self.kraus[i]*rho_0*self.kraus[i] for i in range(len(self.kraus))])
        drho     = [1j*sum([self.kraus[i]*(du[j]*rho_0-rho_0*du[j])*self.kraus[i] for i in range(len(self.kraus))])  for j in range(3)]
        
        D, Vi, snonzero, rnk = circuit.eig(rho, phi_0, n)
        d     = 2**self.n
        npar  = 3
        maskDiag = np.diag(np.ndarray.flatten(np.concatenate((np.ones([rnk,1],dtype = bool),np.zeros([d-rnk,1],dtype = bool)))))
        maskRank = np.concatenate((np.concatenate((np.triu(np.ones(rnk,dtype = bool),1), np.zeros([rnk,d-rnk],dtype = bool)),axis = 1),np.zeros([d-rnk,d],dtype = bool)))
        maskKern = np.concatenate((np.concatenate((np.zeros([rnk,rnk],dtype = bool),np.ones([rnk,d-rnk],dtype = bool)),axis = 1),np.zeros([d-rnk,d],dtype = bool)))

        fulldim = 2*rnk*d-rnk**2

        drhomat = np.zeros((fulldim,npar),dtype=np.complex_)

        for i in range(npar):
            drho[i] = (drho[i].dag()+drho[i])/2
            eigdrho = (Vi.dag())*drho[i]*Vi
            eigdrho = eigdrho.full()
            ak = eigdrho[maskKern]
            ak = ak.reshape((rnk,d-rnk)).transpose()
            ak = ak.reshape((rnk*(d-rnk)))


            row = np.concatenate((eigdrho[maskDiag],np.real(eigdrho[maskRank]),np.imag(eigdrho[maskRank]),np.real(ak),np.imag(ak)))
            drhomat[:,i] =  row

        S   = circuit.SmatRank(snonzero,d, rnk, fulldim)
        S   = (S.transpose().conjugate()+S)/2
        R   = sci.linalg.sqrtm(S) #Rmat(S)

        effdim = R.shape[0]
        idd = np.diag(np.ndarray.flatten(np.concatenate((np.ones((rnk)),2*np.ones((fulldim-rnk))))))

        V = cp.Variable((npar,npar),PSD =True)
        X = cp.Variable((fulldim,npar))

        A = cp.vstack([ cp.hstack([V , X.T @ R.conjugate().transpose()]) , cp.hstack([R @ X , np.identity(effdim) ])])

        constraints = [ cp.vstack([   cp.hstack([cp.real(A),-cp.imag(A)]), cp.hstack([cp.imag(A),cp.real(A)])   ]) >> 0,  
                                     X.T @ idd @ drhomat == np.identity(3)]

        obj = cp.Minimize(cp.trace(V))
        prob = cp.Problem(obj,constraints)
        prob.solve(solver = cp.SCS)#, verbose = True)
        out = prob.value
        if type(out) == None:
            return 1e+5
        else:
            return out
    
    
    def SmatRank(snonzero,d, rnk, dim):

        mask    = np.triu(np.ones([rnk], dtype= bool),1)
        scols   = np.zeros((rnk,rnk))
        for i in range(rnk):
            for j in range(rnk):
                scols[i,j] = np.real(snonzero[j])

        srows   = scols.transpose()
        siminsj = -srows + scols
        siplsj  = scols + srows

        diagS   = np.concatenate((snonzero.transpose(),siplsj[mask].transpose(),siplsj[mask].transpose(),np.matlib.repmat(snonzero.transpose(),2*(d-rnk),1).flatten() ))

        Smat = sci.sparse.spdiags(diagS,0,dim,dim)
        Smat = sci.sparse.csr_matrix(Smat)
        Smat = Smat.todense()
        Smat = np.matrix(Smat, dtype = complex)

        if rnk != 1:
            offdRank = 1j* sci.sparse.spdiags(siminsj[mask],0,int((rnk**2-rnk)/2),int((rnk**2-rnk)/2)).todense()
        else:
            offdRank = 0

        offdKer = -1j*sci.sparse.spdiags(np.matlib.repmat(snonzero,d-rnk,1).flatten(),0,rnk*(d-rnk),rnk*(d-rnk)).todense()   



        Smat[int(rnk+(rnk**2-rnk)/2):int(rnk+(rnk**2-rnk))                        ,int(rnk):int(rnk+(rnk**2-rnk)/2)]                                      =-offdRank;
        Smat[int(rnk):int(rnk+(rnk**2-rnk)/2)                                     ,int(rnk+(rnk**2-rnk)/2):int(rnk+(rnk**2-rnk))]                         = offdRank;
        Smat[int(rnk+(rnk**2-rnk)+rnk*(d-rnk)):int(rnk+(rnk**2-rnk)+2*rnk*(d-rnk)),int(rnk+(rnk**2-rnk)):int(rnk+(rnk**2-rnk)+rnk*(d-rnk))]               =-offdKer;
        Smat[int(rnk+(rnk**2-rnk)):int(rnk+(rnk**2-rnk)+rnk*(d-rnk))              ,int(rnk+(rnk**2-rnk)+rnk*(d-rnk)):int(rnk+(rnk**2-rnk)+2*rnk*(d-rnk))] = offdKer;

        return Smat

    

    def func(phases, *args):
        self, du = args[0], args[1]
        self.sqp = phases
        zo       = qt.basis(2)
        u        = circuit.unitary(self)
        qf_mat   = np.zeros((3,3))
        phi      = qt.tensor([zo]*self.n)
        phi      = u*phi
        dphi     = [-1j*du[i]*phi for i in range(3)]
        #nag_X(phi,dphi, n)
        
        d    = 2**n
        npar = 3

        psidpsi = phi.dag()*dphi;
        pardphi = phi * psidpsi
        #print(dphi, pardphi)
        Lmat    = 2 * (dphi-pardphi)
        #print(Lmat)
        psi = np.zeros(d,dtype= complex)
        for i in range(d):
            psi[i] = phi[i][0][0]

        Lmatt = np.zeros((d,npar),dtype = complex)
        for i in range(d):
            for j in range(npar):
                Lmatt[i,j] = Lmat[j][i] 
        #print(Lmatt)
        V = cp.Variable((npar,npar),PSD =True)
        X = cp.Variable((d,npar),complex = True)

        A = cp.vstack([ cp.hstack([V , X.H ]) , cp.hstack([X , np.identity(d) ])  ])


        constraints = [ cp.vstack([   cp.hstack([cp.real(A),-cp.imag(A)]), cp.hstack([cp.imag(A),cp.real(A)])   ]) >> 0, 
                        cp.real(X.H @ Lmatt) == np.identity(3),
                         ((psi.transpose()).conjugate() @ X) == 0]

        obj = cp.Minimize(cp.trace(V))
        prob = cp.Problem(obj,constraints)
        prob.solve(solver = cp.SCS)#, verbose = True)
        #out = prob.solution
        #print(X.value)
        #print(prob.value)
        out = prob.value
        return out





def pauli_i(phase, theta= 'pi'):
    if theta == 'pi':
        theta = np.pi
    lis = qt.Qobj.expm(-1j*(theta/2)*(phase[0]*sx + phase[1]*sz + phase[2]*sy))
    return lis



In [7]:
n  = 2
gamma = 0.5
kraus = kraus_set(n, gamma)
du = [hamiltonian(sx, n),hamiltonian(sy, n),hamiltonian(sz, n)]
sq  =  [1 for i in range(n*(n+1))]
tq  =  [1 for i in range(n*(n-1))]
a  = circuit(n,1,du, gamma, kraus, init = [sq,tq]) 
u  = a.unitary()

a.qfi

Desired error not necessarily achieved due to precision loss. 2


1.188014874385088

In [145]:
a.sq
zo       = qt.basis(2)
u*qt.tensor([zo]*n)

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.297-0.392j]
 [0.416+0.278j]
 [0.41 +0.277j]
 [0.296+0.417j]]

In [5]:
def genetic(par1, par2):
    extension_rate = 0.001    
    if par1.g == par2.g:
        ch_g = par1.g
        splice_point = np.random.randint(len(par1.sq))
        ch_sq        = np.append(par1.sq[:splice_point], par2.sq[splice_point:])
        ch_sq        = mutate(ch_sq)
        
        splice_point = np.random.randint(len(par1.tq))
        ch_tq        = np.append(par1.tq[:splice_point], par2.tq[splice_point:])
        ch_tq        = mutate(ch_tq)
        

        
        if np.random.random() < extension_rate/par1.g:
            ch_g += 1
            ch_sq.append([np.random.choice(2) for i in range(n*(n+1))])
            ch_tq.append([np.random.choice(2) for i in range(n*(n-1))])
        
        child = circuit(n, ch_g.g, du, init = [ch_sq, ch_tq])
        child = min([child, par1, par2], key = attrgetter('qfim'))
        ###################################################################################
        ###################################################################################
    else:
        par_big, par_small = max([par1, par2], key=attrgetter('g')), min([par1, par2], key=attrgetter('g'))
        
        splice_point_sq = np.random.randint(len(par_small.sq))
        splice_point_tq = np.random.randint(len(par_small.tq))
        ###################################################################################
                
        ch1_sq        = np.append(par1.sq[:splice_point], par2.sq[splice_point:len(par_small.sq)])
        ch1_sq        = mutate(ch1_sq)
        

        ch1_tq        = np.append(par1.tq[:splice_point], par2.tq[splice_point:len(par_small.sq)])
        ch1_tq        = mutate(ch1_tq)
        
        ###################################################################################
        ch2_sq        = np.append(par1.sq[:splice_point], par2.sq[splice_point:])
        ch2_sq        = mutate(ch2_sq)
        

        ch2_tq        = np.append(par1.tq[:splice_point], par2.tq[splice_point:])
        ch2_tq        = mutate(ch2_tq)        
        
        ###################################################################################
        # need to add possibility of extension g += 1 here too
        if np.random.random() < extension_rate/par1.g:
            par_small += 1
            ch1_sq.append([np.random.choice(2) for i in range(n*(n+1))])
            ch1_tq.append([np.random.choice(2) for i in range(n*(n-1))])
            
            par_big += 1
            ch2_sq.append([np.random.choice(2) for i in range(n*(n+1))])
            ch2_tq.append([np.random.choice(2) for i in range(n*(n-1))])            
            
        child1 = circuit(n, par_small.g, du, init = [ch1_sq, ch1_tq])
        child2 = circuit(n, par_big.g  , du, init = [ch2_sq, ch2_tq])
        child  = min([child1, child2, par1, par2], key=attrgetter('qfim'))
    
    
    return child

def mutate(gene, mut_rate):
    #mut_rate = 1/len(gene)
    for i in range(len(gene)):
        prob = np.random.random()
        if prob < mut_rate:
            gene[i] = xor(gene[i],1)
    return gene

In [6]:
##Creating functions for generating input states

def opt_state(n, gamma):
    #create input state by optimizing the quantum circuit
    #n: number of particles
    
    #optimizing the unitary that generates the state using QFIM as the objective function
    kraus = kraus_set(2, gamma)
    du = [hamiltonian(sx, n),hamiltonian(sy, n),hamiltonian(sz, n)]
    a  = circuit(n,1,du, gamma, kraus) 
    u  = a.unitary()
    print(a.qfim) #print the QFIM

    #Genetate input state
    state=qt.tensor([qt.basis(2,0)]*n)

    state=u*state
        
    return state

def GHZ_3D(coefs,n): #returns the superposition of the tensor product of eigenvectors of all pauli matricies
    state = coefs[0]*(qt.tensor([d1[0]]*n) + qt.tensor([d1[1]]*n)) + coefs[1]*(qt.tensor([d2[0]]*n) + qt.tensor([d2[1]]*n)) + coefs[2]*(qt.tensor([d3[0]]*n) + qt.tensor([d3[1]]*n))
    return state.unit()

def product(n):
    state=qt.tensor([qt.basis(2,0)]*n)
    return state


In [7]:
##############Cost Function (and related)
def single_gate(param):
    #param: a vector of the parameters to the Hamitonian
    Ham = param[0]*sx+param[1]*sy+param[2]*sz
    gate = qt.Qobj.expm(-1j*Ham)
    return gate

def var_initialize(n):
    #initializing the paramater for the single gate layers (because it has become too complicated to do 
    #in the cost function without making it hard to read.)
    #n: the number of particles
    
    var = [[[0 for i in range(3)] for j in range(n)] for k in range(n+1)]
    for i in range(n+1): #layer
        for j in range(n): #gate
            for k in range(3): #parameter
                var[i][j][k]=np.random.random();
    
    return var

#define the objective function

def measure_u(seq,var,n):   
    #state is the output state from the quantum channel that depends on the unknown parameters
    #n is the number of particles   
    #seq: the sequence to turn the layers on and off
    #var: the variables for the single-qubit gates
    
    #Create the circuit according to the sequence given
    
    for i in range(2*n+1):
        if seq[i]==1: #if the layer is turned on
            if np.mod(i,2)==0:
                #Single-qubit layer
                for m in range(n):
                    if m == 0:
                        gate = single_gate(var[(int)(i/2)][m])
                    else:
                        gate = qt.tensor(gate,single_gate(var[(int)(i/2)][m]))             
                #print("single gate")                
            elif np.mod(i,2)==1:
                #Two-qubit layer
                #detemine control qubit
                control = (int)(i/2);
                for m in range(n):
                    if m != control:
                        #Create CNOT for m
                        if m == 0:
                            gate = qt.cnot(n,control,m)
                        else:
                            gate = gate*qt.cnot(n,control,m)
                #print("two-gate")
            else:
                #do nothing
                print("something's wrong")
        else: #if the layer is turned off
            gate=qt.tensor([ide]*n)

        #Operating the layer onto previous layers
        if i == 0:
            output_gate=gate
        else:
            output_gate=gate*output_gate
        
    return output_gate

def prob_pure(state,x,n):
    #Calculate the probability of output x from pure state
    #state: pure state
    #x: classical output of the measurement -- bitstring
    #n: number of particles

    ##create the ket for x
    #turn string to bit
    x_array=[0 for i in range(n)]
    
    j=0
    for i in x:
        x_array[j]=i
        j=j+1
    
    for m in range(n):
        bit=(int)(x_array[m])
        if m ==0:
            qubit=qt.basis(2,bit)
        else:
            qubit=qt.tensor(qubit,qt.basis(2,bit))

    P=((qubit.dag()*state).norm())**2
    
    return P

def prob_mixed(state,x,n):
    #Calculation the probability of output x from mixed state
    #state: mixed state
    #x: classical output of the measurement -- bitstring
    #n: number of particles
    
    #create projective measurement matrix for x
    x_array=[0 for i in range(n)]    
    j=0
    for i in x:
        x_array[j]=i
        j=j+1
    #qt.Qobj(x_array,dims=[[2]*n,[1]*n])
    for m in range(n):
        bit=(int)(x_array[m])
        if m ==0:
            qubit=qt.basis(2,bit)
        else:
            qubit=qt.tensor(qubit,qt.basis(2,bit))
    
    rho=qubit*qubit.dag()*state
    
    P=rho.tr()
    
    return P


def div_phase(phase, n):
    #phase: phase vector as input
    #n : the number of particles
    #the output is a three-matrices vector for differentiation in each x,y,z
    #this version of the code assume no noise just yet
    
    #declare constants
    phi=np.sqrt(phase[0]**2+phase[1]**2+phase[2]**2)
    
    if phi != 0:
        nx=phase[0]/phi
        ny=phase[1]/phi
        nz=phase[2]/phi
        A=(np.sin(phi)**2)/(phi)
        B=np.sin(2*phi)/(4*phi)
    else:
        nx = 1
        ny = 1
        nz = 1
        A=0
        B=0.5
    ##Let's start with dUz
    #single particle    
    temp=(1/2+B)*sz+(1/2-B)*((nz**2-nx**2-ny**2)*sz+2*nz*(nx*sx+ny*sy))-A*(nz*sx-nx*sy)
    #tensor product for N particles
    dUz=sum( [qt.tensor([ide if i !=j else temp for i in range(n) ]) for j in range(n)])
    dUz=-1j*opr*dUz
    
    ##dUy
    #single particle
    temp=(1/2+B)*sy+(1/2-B)*((ny**2-nx**2-nz**2)*sy+2*ny*(nx*sx+nz*sz))-A*(nx*sz-nz*sx)
    #tensor product
    dUy=sum( [qt.tensor([si if i !=j else temp for i in range(n) ]) for j in range(n)])
    dUy=-1j*opr*dUy
    
    ##dUx
    #single particle
    temp=(1/2+B)*sx+(1/2-B)*((nx**2-ny**2-nz**2)*sx+2*nx*(ny*sy+nz*sz))-A*(nz*sy-ny*sz)
    #tensor product
    dUx=sum( [qt.tensor([si if i !=j else temp for i in range(n) ]) for j in range(n)])
    dUx=-1j*opr*dUx
    
    du=(dUx,dUy,dUz)
    return du

def cost_func(x0,state,n): #finally a cost function??
    #x0 here is a stand in for the input to the optimization, in this case, there are two elements, the sequence and the var array
    #state is the output state from the quantum channel that depends on the unknown parameters
    #n is the number of particles
    
    seq=x0[0]
    var=x0[1]
    
    F=[[0,0,0],[0,0,0],[0,0,0]] #F_xi,F_yi,F_zi
    
    Ucom=[[0,0],[0,0],[0,0]] #array to keep the combination of U,dU and input state
    
    du=div_phase(phase,n)
    
    measure_gate=measure_u(seq,var,n)
    
    for i in range(3):
        Ucom[i][0]=measure_gate*du[i]*state*opr.dag()*measure_gate.dag()
        Ucom[i][1]=measure_gate*opr*state*du[i].dag()*measure_gate.dag() 
        
    ##Time to calculate F_ij
    for i in range(3):
        for j in range(3):
            for x in range(2**n): #calculate over x bitstring
                b=format(x, '0'+str(n)+'b')
                P=prob_mixed(measure_gate*state*measure_gate.dag(),b,n)
                if P != 0 :
                    F[i][j]=F[i][j]+4/P*np.real(prob_mixed(Ucom[i][0],b,n))*np.real(prob_mixed(Ucom[j][0],b,n))

    #Cost function that calculate the trace of the inverse Fisher information
    #This cost function require minimization
        
    try:
        invF=np.linalg.inv(F)
        absF=np.matrix.trace(invF)
        
    except:
        absF=1000000000
        
    if absF < 0:
        absF=1000000000
        
    return absF


In [8]:
#########Noise model############

def Kraus_gen(gamma,n):
    #This function creates all possible combination and permutation of N-qubit Kraus operator from single-qubit Kraus operator
    #e1 and e2 are Kraus operators for single-qubit for noise
    #n is the number of particles

    e1 = np.sqrt((1+np.sqrt(1-gamma))/2)*ide
    e2 =np.sqrt((1-np.sqrt(1-gamma))/2)*sz

    
    K_array=[0 for i in range(2**n)]
    x_array=[0 for i in range(n)]
    
    for x in range(2**n):
        #create the array of bit to create the operator
        b=format(x, '0'+str(n)+'b')
        
        j=0
        for i in b:
            x_array[j]=i
            j=j+1
            
        for m in range(n):
            bit=(int)(x_array[m])
            if m == 0:
                if bit == 0:
                    temp=e1
                else:
                    temp=e2
            else:
                if bit == 0:
                    temp=qt.tensor(temp,e1)
                else:
                    temp=qt.tensor(temp,e2)
                    
        K_array[x]=temp
        
    return K_array

def noisy_state(state0,n,gamma):
    
    
    K_array=Kraus_gen(gamma,n)
    ##we are still missing the weights and normalization to the Kraus operator
    
    state=sum(K_array[i]*state0*K_array[i].dag() for i in range(2**n))
    
    return state

In [28]:
##Put the state through the quantum channel
#Still has to put in proper noise model
n=2
phase=[0.0,0.0,0.0]
gamma = 0.9

Ham=phase[0]*sx+phase[1]*sy+phase[2]*sz
Uni=qt.Qobj.expm(-1j*Ham)

state=opt_state(n, gamma)
#state=GHZ_3D([1,1,1],n)
#state=product(n)

print(state*state.dag())

opr=qt.tensor([Uni]*n)        
state=opr*state
state=state/np.sqrt(state.norm())

##Add noise model
#turn state to mixed state
state0=state*state.dag()
state = noisy_state(state0,n,gamma)

state0=state

print(state)

0.9661319026529429
Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.572+0.j    0.   +0.j    0.123+0.479j 0.   +0.j   ]
 [0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j   ]
 [0.123-0.479j 0.   +0.j    0.428+0.j    0.   +0.j   ]
 [0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j   ]]
Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.572+0.j    0.   +0.j    0.039+0.152j 0.   +0.j   ]
 [0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j   ]
 [0.039-0.152j 0.   +0.j    0.428+0.j    0.   +0.j   ]
 [0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j   ]]


In [29]:
###################OPTIMIZATION--OF--MEASUREMENT###################
#constants
T = 200 #interation/generation before stopping
Cr = 0.6 #DE Cross over parameter
f = 0.6 #DE scaling parameter
pop_size = 50 #DE population size
family_size=3 #size of family
seq_size=2*n+1
var_bound=[[0,2*np.pi],[0,2*np.pi],[0,2*np.pi]]
### Initialize population
#initializing the on/off sequence -- actually, I don't think we need to separate the single- and two-qubit gate string
seq_pop=[[np.random.choice(2) for i in range(seq_size)]for j in range(pop_size)]
#initializing the starting variables for single-qubit gates (in the first layer only)
var_pop = [var_initialize(n) for j in range(pop_size)]

### Initialize fitness
fitness=[0 for i in range(pop_size)]

for i in range(pop_size):
    state=state0
    x0=(seq_pop[i],var_pop[i])
    fitness[i]= cost_func(x0,state,n)
    
print(fitness)

seq_off=[[0 for i in range(seq_size)]for j in range(pop_size)]
var_off = [var_initialize(n) for j in range(pop_size)]
fitness_off=[0 for i in range(pop_size)]

seq_temp=[0 for i in range(seq_size)]
var_temp = [[[0 for i in range(3)] for j in range(n)] for k in range(n+1)]
### Start optimizing
for t in range(T):
    ##generate offspring
    for i in range(pop_size):
        #choose family
        family=[np.random.randint(0,pop_size) for i in range(family_size)]
        if pop_size>2*family_size:
            while family[0]==i:
                family[0]=np.random.randint(0,pop_size)
            while family[1]==family[0] or family[1]==i:
                family[1]=np.random.randint(0,pop_size)
            while family[2]==family[1] or family[2]==family[0] or family[2]==i:
                family[2]=np.random.randint(0,pop_size)
        #create the offspring
        #--sequence array--
        for j in range(seq_size):
            coin=np.random.random()
            if coin<=Cr:
                seq_temp[j]=seq_pop[family[0]][j]+(seq_pop[family[1]][j]-seq_pop[family[2]][j])
                seq_temp[j]=seq_temp[j]%2
        #--variables array--
        for j in range(n+1):
            for k in range(n):
                for l in range(3):
                    coin=np.random.random()
                    if coin<=Cr:
                        var_temp[j][k][l]=var_pop[family[0]][j][k][l]+f*(var_pop[family[1]][j][k][l]-var_pop[family[2]][j][k][l])
                    if var_temp[j][k][l]<var_bound[l][0]:
                        var_temp[j][k][l]=var_bound[l][0]
                    if var_temp[j][k][l]>=var_bound[l][1]:
                        var_temp[j][k][l]=var_bound[l][1]

        seq_off[i]=seq_temp
        var_off[i]=var_temp
        #calculate offspring's fitness
        state=state0
        x0=(seq_temp,var_temp)
        fitness_off[i]= cost_func(x0,state,n)
    ##Here we're done creating offspring and computing fitness
    
    ##select population for the next generation
    for j in range(pop_size):
        if fitness_off[j]<fitness[j]:
            fitness[j]=fitness_off[j]
            seq_pop[j]=seq_off[j]
            var_pop[j]=var_off[j]
    
###Once T runs out, we select the best solution we can find
solution=[min(fitness),seq_pop[fitness.index(min(fitness))],var_pop[fitness.index(min(fitness))]]

print(solution)

[1.859082625039374e+17, 42.29463668230269, 1000000000, 4723.120193781395, 38.72425375622138, 1000000000, 19.536869187580507, 1000000000, 1817.5402282724478, 2408.432677167077, 4.1592630213494355e+17, 1891.0951401891211, 23.423263353334573, 1000000000, 448.01259779959514, 12.538818251219112, 518.6590084742597, 20.112977292962203, 1000000000, 196.07328767154831, 108579.02763480895, 37.661445032208434, 5670515038992375.0, 1000000000, 4608.4339474121225, 15.185431676796782, 1.3617127430340051e+17, 1000000000, 154.32542827826262, 116.8009600483741, 18.425829722104766, 53.646358520158, 17.212539808824033, 281.2683570050049, 73.33997690685948, 1000000000, 56.390565134607826, 1240.2780805306961, 11.79438869321171, 1000000000, 98.34047371314341, 7730208900733960.0, 4851.214170990423, 1000000000, 140.40322883245574, 2.793074752063261e+16, 2830.622809862754, 1000000000, 1000000000, 78.3920167432783]
[4.193476610741359, [1, 1, 1, 1, 0], [[[0.7014187655423413, 0.49128256418572025, 0.912987455180438

GHZ state

gamma = 0

[437.7043021800269, 1000000000, 2.642898180241575, 1.6229636690810534, 436.9185885316472, 4.216440014082267, 4.7938533217819685, 2.1578501004978614, 16.920940897898028, 2.767270112960071, 40.768123686187764, 6.447068299752538, 1.1286863072032376, 1000000000, 1000000000, 11.633082092919038, 1000000000, 2.547059784676886, 17.851988884460006, 1.7213392476294538, 4.335230683766161, 5.687137202779822, 256.7705514611661, 1000000000, 1000000000, 95.24484071767705, 1000000000, 1.9369508180911101, 270.4433916519325, 1.8774277810098792, 135.30418023892102, 1.216785229375193, 1.813901619922698, 2.2768321733280215, 1000000000, 237.77910704254418, 3.6235425820771616, 6.101612915722992, 72.54544384420092, 1000000000, 5.042017760058273, 35.16649442772674, 1.9784314940413228, 1.3311368846667513, 4.61440012297239, 1.6198719934207315, 1.5473111795502341, 1000000000, 1.6578473719760074, 6.603414331001665]
[0.99695350129001, [1, 0, 0, 1, 1], [[[0.9228649150209723, 0.4881303743465588, 0.6419510537530863], [0.575812505501503, 0.9158306457690508, 0.1312069196765363]], [[0.5112225336058303, 0.4075094425839694, 0.9198825591728167], [0.8357040611949176, 0.7638604609169127, 0.5992798081096057]], [[0.2551610106375518, 0.49236246050907123, 0.7415266843942074], [0.5556265092535791, 0.30781141524214184, 0.6935037073002843]]]]

gamma = 0.2

[56.346830482901, 1000000000, 4.911603473281358, 1000000000, 1000000000, 3.562446404224361, 6.598193605339908, 1.2608315674916502, 1.0992924252283525, 1188.3824968065367, 2.6964635434104673, 1.3566219689422119, 2486.368344713184, 2.9674340069678116, 1.2539489641282116, 25.463293839545337, 40.44881368832363, 6.960693641837197, 1.177835850139765, 13.74681577298911, 1000000000, 5.991726487572616, 143.13204567641318, 111.06110319422892, 4.848074696765963, 1000000000, 9.028816919582145, 1.358967038387635, 1353.3411305456848, 10.492083015223997, 2.1248121966333793, 62.4316077334167, 7.71291222937702, 1000000000, 7448.577158650856, 29.943103182183346, 8.235480627301676, 1.1307552710812678, 50.59149345613497, 1.3369395071558379, 1000000000, 287.7195044227177, 19.97237405042641, 50.60118797898167, 1.3055181819934671, 1.7756505142019088, 7.100971530253586, 1.383460408759365, 1.4749728999062686, 1.3932786609636318]
[1.0796596329536798, [0, 1, 1, 0, 1], [[[0.1915454820522665, 0.41254109088604474, 0.09506004607575247], [0.7634622263641047, 0.4609695538521384, 0.839006551164115]], [[0.5599597022393396, 0.2589205258650663, 0.1681422844425261], [0.7946775473883894, 0.325742025302298, 0.25906678863254484]], [[0.2608183785311978, 0.22095759833347128, 0.9222612620589646], [0.5981595560422149, 0.46790799916622694, 0.7593214149031623]]]]

gamma = 0.4

[1976.5804265742165, 2.678163782483919, 1.4939087442084789, 47.71613987483518, 1251.0467613009193, 240.3898661666811, 1000000000, 6.035911712763919, 1.9967038653331057, 9.825381047209067, 1000000000, 2.7973702409442667, 117.23411454696972, 1000000000, 332.049268085929, 1000000000, 2.134722912883763, 5.094799974752829, 4.060578178844131, 1.8200106933821627, 7.989005109056404, 3.123942162647796, 2.8426129507246123, 1000000000, 10.043445977185813, 2.3795042689803854, 3.0235542126092247, 5.886546781260747, 3.8654568430864216, 2.040012037390924, 1.9161566898554252, 55.77105798367601, 28239.921863368705, 8.318138356373325, 1000000000, 2.733132229538704, 4.356876378655166, 2.290274749381071, 1.3687001620113457, 1000000000, 1000000000, 9.846461189360204, 3.8957160026965587, 1000000000, 578.9633193368875, 1.2459875936370828, 13.799630909192523, 8.349576172988739, 1.3581859554040832, 1000000000]
[1.2344037183404086, [0, 1, 1, 0, 1], [[[0.6948777619606302, 0.7237913573576086, 0.31323298080329853], [0.5066436064691936, 0.3912888848134364, 0.8697763928282114]], [[0.02023773226198533, 0.6960309884183552, 0.465218612626744], [0.25583410138317564, 0.8353317320216276, 0.7171170651560352]], [[0.10897864775987098, 0.4858852562352307, 0.2621468139364652], [0.652806644855247, 0.4166632235685023, 0.08424809657930743]]]]

gamma = 0.6

[9.265052692928574, 5.594469902269906, 153.5418384245594, 57.70434041150898, 158.36070165676483, 227.0359212593112, 1000000000, 16.737140483805355, 2.0630111338595585, 1000000000, 259.44934065636386, 2.4962742226610306, 6.90523019299238, 235.32411208656623, 89.49092822958968, 10.939338541789827, 273.8523351710292, 22.938845103503144, 1000000000, 13.715618018839004, 32.37882870839355, 1.947701306571326, 6012.768062065102, 3.283569618816108, 1.9111776927945159, 2.7124491529823023, 23.393392240322502, 11.580510452300974, 170.664575601696, 9.565550590643587, 1000000000, 15.290665860147636, 23.36738350556607, 4.874429163600073, 3.367446845933572, 99.79469209597073, 21.142752025684416, 77.42100082001473, 3.5637824455173686, 6.588524927590818, 36.488741140379986, 12877.7575898845, 702.2202854078124, 3.026159979363262, 1.9898511442882787, 46.443023753827134, 44.90106083578384, 9.4575153588819, 1.7785021638065204, 3.6017517546687214]
[1.7082669697423825, [0, 1, 0, 1, 1], [[[0.19224233697857807, 0.7746321019123323, 0.31699438379132905], [0.101487195798591, 0.5642421494193245, 0.8994672777098329]], [[0.08281679363516314, 0.2629061817643293, 0.6111478814627336], [0.5006466679799538, 0.42305641487639256, 0.8135588928504707]], [[0.7913837288329768, 0.6775105192021008, 0.35059062812101116], [0.8728803075539742, 0.4821382168014513, 0.5508802343714189]]]]

gamma = 0.8

[5.313275828362694, 8.278923197365952, 22.01475665605858, 16.225649401982142, 148.0620610361545, 101.51855203580081, 9.159032338172114, 13.678671044689601, 1959.6112128005466, 46.364040716688585, 7.796467393036389, 29.17730298356391, 17.938947790695753, 6.8051344936643225, 1000000000, 1000000000, 20.762835285568855, 26.259732235461556, 6.2540654594725, 23.066088834383176, 24.052016157320217, 31.590510533345174, 46.130322780426425, 25.290816289863592, 43.85419470731323, 43.42541406402083, 1000000000, 1000000000, 1000000000, 103031.33697345947, 91.82526119906109, 1000000000, 6.343362477997569, 1000000000, 74.05156220280568, 5.789533230690733, 16.562874521553567, 2120.882165834809, 145.7999988462268, 18.52207335848306, 36464.3740846139, 2974.5941571796507, 6.955759040087873, 4.94746021109627, 1000000000, 111.36148370039766, 47.92859689653639, 6.511840871046026, 71.50772972069754, 109.95613778676089]
[4.900622318544019, [0, 1, 1, 1, 1], [[[0.21500482513851693, 0.6460190670508673, 0.24099930057220886], [0.5195161512003372, 0.636397488301052, 0.8772555028396443]], [[0.38829183019846414, 0.8197540399255648, 0.2914203028651863], [0.7800148773935017, 0.5601260769154836, 0.45144092306420947]], [[0.8812546222477402, 0.758402681455998, 0.1883168195987761], [0.38961544802550263, 0.8223983353262875, 0.7013837120217158]]]]



Optimized State

gamma =0 

[1.7386040633530792e+16, 1000000000, 239.9425358559751, 1000000000, 5.176287869510151, 17.508978379683466, 5.747520417142868e+17, 1000000000, 2.2818078301256985, 1000000000, 13.817459579013132, 1000000000, 9.704136748406674e+16, 1000000000, 2.3031931481600303, 1000000000, 2.647939974885801, 6.153363297852209, 1.4838658899527978e+16, 1405.4145654647737, 1000000000, 19.617910463717088, 6.716038240583373, 1000000000, 7.16167492896382, 1.740542725319738, 11535.413181407046, 1.1572982067833508e+16, 4.893382579817027, 1000000000, 2.3097064350953307, 10.79226436020506, 2.1543688778914905, 3.3141104557790544, 1000000000, 40.78914542805595, 1.280333536120465e+16, 3.189595036853741, 14.075070119257855, 41.55959332969898, 2.460424024008908e+16, 1.7543673811786045, 2.002487332888903, 1000000000, 2.1404160754334125, 2.483553746415737, 21.30931192303896, 21.145929600337187, 2.0569074478989537, 317.7138388225548]
[1.039474795912042, [0, 1, 0, 1, 1], [[[0.23965086589732665, 0.9545630151590292, 0.43973684829251136], [0.5570427790037009, 0.6214348298691174, 0.5853800546777707]], [[0.8184647624620036, 0.7748239119060886, 0.6913183921941941], [0.7844973979190414, 0.11892172897746422, 0.4017762759294833]], [[0.262676132418465, 0.15133200916648748, 0.12476404020757081], [0.07441788657630921, 0.6931905523667965, 0.45312670535268107]]]]

gamma = 0.1 

[9.01570734786725, 2.3333024905535362, 2.1876716046843114, 8.69652156908569, 7.038460458808448, 22.335930443579816, 590.4553566093887, 25.796057163077844, 1.0508319352110148e+16, 590.1750710329062, 25.347859072606088, 1000000000, 12.967006648016195, 1000000000, 39.210067970757514, 1000000000, 1000000000, 4.582571191178382, 4.697031702375828e+17, 10.608786207833381, 1000000000, 5.464640640031635, 1.9884235220389583, 8.010772521827603, 1000000000, 335.20164874986654, 1000000000, 9.496233573910573, 1000000000, 1000000000, 59.55862064357018, 61.07478239152558, 18.925038143914556, 1000000000, 2.375471726209733, 4485.7635735176445, 1000000000, 5935.090318182484, 19.57192966866336, 1000000000, 5.174124131909641, 2.4625143230683646, 4.935435185708529e+16, 77.63347795987995, 7478666030561392.0, 1000000000, 1.5247134167179277e+17, 2.386224595500777, 1000000000, 92.99502469738113]
[1.2385718946113888, [0, 0, 0, 1, 1], [[[0.9299191859546776, 0.6480794138497359, 0.1310710685879274], [0.9191031373824892, 0.5157559537802985, 0.011620158856625595]], [[0.8451717058751705, 0.35634189743022504, 0.6852661850045965], [0.9602600441332954, 0.6881623943191967, 0.35187947529744346]], [[0.8661414921590236, 0.5996528163720672, 0.7534779243810388], [0.3289006102508536, 0.0960440975968695, 0.30987917138514454]]]]

gamma = 0.2

[206.1637768665874, 33.41000895313542, 25.148665273248113, 1000000000, 29.764562794251695, 9.125315034232656, 1000000000, 1000000000, 13.013775536631893, 1.0656866634075842e+16, 1000000000, 49.850058705631255, 1000000000, 142.9542399524828, 1000000000, 11.183261463907264, 4.8853559284635475, 1000000000, 5.026072963209463, 3.2943236547509236, 1000000000, 1000000000, 5.520214569206058, 1535.109142776895, 5.140924163983843, 6.4392049682002845, 3.976793852062184, 1000000000, 13.507829530805195, 1576.4194777896325, 2.6420814292592967, 6.286143023304694, 1000000000, 5676543228388226.0, 4.776809128335761, 1.3575413650456706e+16, 1000000000, 1.1247082980096146e+16, 7.055401337516383, 561.6865305097674, 4.332161759568198, 1.1252431877214436e+16, 236.03702603684098, 23.682743260308435, 1000000000, 1000000000, 44.4440653093077, 44.5976021974834, 4.974822252781619, 2.024169204607497]
[1.6279265203399667, [0, 1, 0, 0, 1], [[[0.5378988717144042, 0.275939177874071, 0.3333364085936781], [0.9376468436537393, 0.14800547292701083, 0.8806079935799079]], [[0.7819196980508046, 0.5191478616916722, 0.9593615487090378], [0.5830439163974648, 0.7246154842550517, 0.3626132329794362]], [[0.0, 0.7461896963622755, 0.8499215240196301], [0.3668519720891599, 0.4309074502817952, 0.12796782415975763]]]]

gamma = 0.4
[20.621095337449425, 16.458931833645607, 9752042131526606.0, 2.6455999182100428e+16, 31.6252749403536, 3.1784398873757548e+16, 1000000000, 7.332853873038808, 7.113976842635553, 1.024485309428937e+16, 2137.725539214508, 1000000000, 11.678318476304625, 23.088158791838346, 1000000000, 3.7659266491770644, 103.74530711440003, 1000000000, 76.51651671160633, 9755185968678766.0, 5171575127471222.0, 2.5022690336840627, 1000000000, 8247259926780068.0, 6.113439312118807, 82.47265319304101, 8.60053780102743e+16, 33.68645658259804, 22.631974102057114, 1.7695324774654004e+16, 1000000000, 11.93059139959789, 7.116227422745514, 560.8414094570844, 3786.2062108924215, 14.284373404814025, 1000000000, 1000000000, 9.554309328592478, 573.3087004378605, 1000000000, 1000000000, 1000000000, 1000000000, 34.78719387081691, 8.797850314070622e+16, 1000000000, 284.5655489031793, 9.949527130517941, 1000000000]
[2.5022690336840627, [1, 0, 0, 1, 1], [[[0.9896834533724865, 0.593504392156554, 0.8156025472386078], [0.7346068938185285, 0.48895607808321384, 0.5322654663486828]], [[0.521580876869997, 0.21847480521818852, 0.6858802822935064], [0.8410192737468458, 0.4592779862173664, 0.8089734927888961]], [[0.49217907667612204, 0.474908248758526, 0.3394275533732959], [0.4777115385355978, 0.44241630773397345, 0.9535989142496653]]]]

gamma=0.6

[1000000000, 12.523342063732112, 41.27162606203052, 1.3719442225812862e+16, 1000000000, 10.455224966031457, 7.507171781296173, 87.73460639301433, 7.259271464705135, 1000000000, 1000000000, 1000000000, 1000000000, 12.782997269900198, 25.53627219421746, 71.96060053068285, 22.2247758197906, 43.000164486301635, 18.811683139737717, 2576.514938131272, 1.272348732036541e+16, 21.68499168388749, 101.06513687524213, 1000000000, 2.6964323497214413, 146.04821610678724, 1000000000, 2.8182515838120916e+16, 12.059041514361098, 1.3206518272953461e+17, 255.39819716230235, 5.648658629991656, 21.59916658587333, 1000000000, 1000000000, 9547261385609264.0, 8.980001227836233, 7.224225505671148, 1000000000, 11.630685881102782, 8.332948486431448e+16, 20.039480989048883, 5.657644726281682, 25.68264577195503, 2.2453200446895622e+17, 32.22692391040838, 25.768085411552356, 6.417021529416374, 185.56386294143024, 1000000000]
[2.455239580360055, [0, 1, 1, 0, 1], [[[0.8632205053830995, 0.4131046509617228, 0.5516679340254902], [0.6463322510401542, 0.07251974279857654, 0.05861086649605152]], [[0.22384885639707197, 0.3536294786352505, 0.056874273707794076], [0.13400268292878925, 0.23233643418952307, 0.18488613644273433]], [[0.9239094069654193, 0.5543687862527599, 0.12152660329901865], [0.0746274316157236, 0.31495545938930136, 0.095513918761154]]]]


gamma = 0.8

[280.2359929392721, 305.27908647504466, 9.630985045870288, 7.239365141517575, 13.550642674742233, 4257.651448448129, 16.849713047719902, 67.14185701164361, 519.3142076585923, 8.065480697359366, 7.272636148084933, 1000000000, 28.438751748254617, 28.938611383160506, 1629.0673757630875, 14.895293797999166, 9.74407918692594, 1335.8590218702027, 129.04812751626346, 9.459711513253051, 322.43243341581467, 6.523789019705763, 200.56019253018576, 1000000000, 17.310335563047477, 19.316151256166048, 125.88386061573931, 1405.8293750857506, 24876.006313708076, 15.156882036585229, 5.678596019354858, 28.98801503711344, 13.195367651823677, 132.4847770093776, 204.1017033836281, 3478.110888184411, 55546.861646763035, 7.38515669520239, 1662.405675858976, 6.538524545688249, 350.28850738761093, 21.119556486636117, 1000000000, 7.4358646744902615, 111.13198272882875, 6.284111252844941, 1000000000, 1000000000, 65.14570978286339, 196.27545942112346]
[4.435740816928238, [0, 0, 1, 0, 1], [[[0.5148703672168582, 0.694157504833119, 0.4240788260040881], [0.6592637089064688, 0.9306570475216189, 0.48938740250902446]], [[0.45654443938210587, 0.5478111543119432, 0.1639995114969517], [0.7947777605368499, 0.4122567002441233, 0.3505218985565649]], [[0.32720136574175895, 0.9525015079122346, 0.17263286497362806], [0.16591611222711, 0.2277948494273434, 0.7647317744822155]]]]



In [10]:
n=2
du = [hamiltonian(sx, n),hamiltonian(sy, n),hamiltonian(sz, n)]
a = circuit(n,1,du) 
u = a.unitary()

a.qfim

inf
inf


TypeError: unsupported operand type(s) for -: 'NoneType' and 'NoneType'